In [1]:
# Based on the paper: Inductive Matrix Completion Based on Graph Neural Networks
import os
import scipy.sparse as sp

import numpy as np
import pandas as pd
import torch as th
import torch
import time

import dgl 
from dgl.data.utils import download, extract_archive, get_download_dir
from refex import extract_refex_feature
import argparse

import math 
import torch as th 
import torch.nn as nn
import torch.nn.functional as F

from utils import MetricLogger, get_neighbor_nodes_labels, subgraph_extraction_labeling, load_official_trainvaltest_split

from dgl.nn.pytorch import RelGraphConv

Using backend: pytorch


In [2]:
def load_official_trainvaltest_split(dataset, testing=False, rating_map=None, post_rating_map=None, ratio=1.0):
    """
    Loads official train/test split and uses 10% of training samples for validaiton
    For each split computes 1-of-num_classes labels. Also computes training
    adjacency matrix. Assumes flattening happens everywhere in row-major fashion.
    """

    sep = '\t'

    # Check if files exist and download otherwise
    files = ['/u1.base', '/u1.test', '/u.item', '/u.user']
    fname = dataset
    data_dir = 'data/' + fname


    dtypes = {
        'u_nodes': np.int32, 'v_nodes': np.int32,
        'ratings': np.float32, 'timestamp': np.float64}

    filename_train = 'data/' + dataset + '/u1.base'
    filename_test = 'data/' + dataset + '/u1.test'

    data_train = pd.read_csv(
        filename_train, sep=sep, header=None,
        names=['u_nodes', 'v_nodes', 'ratings', 'timestamp'], dtype=dtypes)

    data_test = pd.read_csv(
        filename_test, sep=sep, header=None,
        names=['u_nodes', 'v_nodes', 'ratings', 'timestamp'], dtype=dtypes)

    data_array_train = data_train.values.tolist()
    data_array_train = np.array(data_array_train)
    data_array_test = data_test.values.tolist()
    data_array_test = np.array(data_array_test)

    if ratio < 1.0:
        data_array_train = data_array_train[data_array_train[:, -1].argsort()[:int(ratio*len(data_array_train))]]

    data_array = np.concatenate([data_array_train, data_array_test], axis=0)

    u_nodes_ratings = data_array[:, 0].astype(dtypes['u_nodes'])
    v_nodes_ratings = data_array[:, 1].astype(dtypes['v_nodes'])
    ratings = data_array[:, 2].astype(dtypes['ratings'])
    if rating_map is not None:
        for i, x in enumerate(ratings):
            ratings[i] = rating_map[x]

    u_nodes_ratings, u_dict, num_users = map_data(u_nodes_ratings)
    v_nodes_ratings, v_dict, num_items = map_data(v_nodes_ratings)

    u_nodes_ratings, v_nodes_ratings = u_nodes_ratings.astype(np.int64), v_nodes_ratings.astype(np.int32)
    ratings = ratings.astype(np.float64)

    u_nodes = u_nodes_ratings
    v_nodes = v_nodes_ratings

    neutral_rating = -1  # int(np.ceil(np.float(num_classes)/2.)) - 1

    # assumes that ratings_train contains at least one example of every rating type
    rating_dict = {r: i for i, r in enumerate(np.sort(np.unique(ratings)).tolist())}

    labels = np.full((num_users, num_items), neutral_rating, dtype=np.int32)
    labels[u_nodes, v_nodes] = np.array([rating_dict[r] for r in ratings])

    for i in range(len(u_nodes)):
        assert(labels[u_nodes[i], v_nodes[i]] == rating_dict[ratings[i]])

    labels = labels.reshape([-1])

    # number of test and validation edges, see cf-nade code

    num_train = data_array_train.shape[0]
    num_test = data_array_test.shape[0]
    num_val = int(np.ceil(num_train * 0.2))
    num_train = num_train - num_val

    pairs_nonzero = np.array([[u, v] for u, v in zip(u_nodes, v_nodes)])
    idx_nonzero = np.array([u * num_items + v for u, v in pairs_nonzero])

    for i in range(len(ratings)):
        assert(labels[idx_nonzero[i]] == rating_dict[ratings[i]])

    idx_nonzero_train = idx_nonzero[0:num_train+num_val]
    idx_nonzero_test = idx_nonzero[num_train+num_val:]

    pairs_nonzero_train = pairs_nonzero[0:num_train+num_val]
    pairs_nonzero_test = pairs_nonzero[num_train+num_val:]

    # Internally shuffle training set (before splitting off validation set)
    rand_idx = list(range(len(idx_nonzero_train)))
    np.random.seed(1234)
    np.random.shuffle(rand_idx)
    idx_nonzero_train = idx_nonzero_train[rand_idx]
    pairs_nonzero_train = pairs_nonzero_train[rand_idx]

    idx_nonzero = np.concatenate([idx_nonzero_train, idx_nonzero_test], axis=0)
    pairs_nonzero = np.concatenate([pairs_nonzero_train, pairs_nonzero_test], axis=0)

    val_idx = idx_nonzero[0:num_val]
    train_idx = idx_nonzero[num_val:num_train + num_val]
    test_idx = idx_nonzero[num_train + num_val:]

    assert(len(test_idx) == num_test)

    val_pairs_idx = pairs_nonzero[0:num_val]
    train_pairs_idx = pairs_nonzero[num_val:num_train + num_val]
    test_pairs_idx = pairs_nonzero[num_train + num_val:]

    u_test_idx, v_test_idx = test_pairs_idx.transpose()
    u_val_idx, v_val_idx = val_pairs_idx.transpose()
    u_train_idx, v_train_idx = train_pairs_idx.transpose()

    # create labels
    train_labels = labels[train_idx]
    val_labels = labels[val_idx]
    test_labels = labels[test_idx]

    if testing:
        u_train_idx = np.hstack([u_train_idx, u_val_idx])
        v_train_idx = np.hstack([v_train_idx, v_val_idx])
        train_labels = np.hstack([train_labels, val_labels])
        # for adjacency matrix construction
        train_idx = np.hstack([train_idx, val_idx])
    
    class_values = np.sort(np.unique(ratings))

    # make training adjacency matrix
    rating_mx_train = np.zeros(num_users * num_items, dtype=np.float32)
    if post_rating_map is None:
        rating_mx_train[train_idx] = labels[train_idx].astype(np.float32) + 1.
    else:
        rating_mx_train[train_idx] = np.array([post_rating_map[r] for r in class_values[labels[train_idx]]]) + 1.
    rating_mx_train = sp.csr_matrix(rating_mx_train.reshape(num_users, num_items))

    if dataset =='ml-100k':

        # movie features (genres)
        sep = r'|'
        movie_file = 'data/' + dataset + '/u.item'
        movie_headers = ['movie id', 'movie title', 'release date', 'video release date',
                         'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
                         'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                         'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                         'Thriller', 'War', 'Western']
        movie_df = pd.read_csv(movie_file, sep=sep, header=None,
                               names=movie_headers, engine='python')

        genre_headers = movie_df.columns.values[6:]
        num_genres = genre_headers.shape[0]

        v_features = np.zeros((num_items, num_genres), dtype=np.float32)
        for movie_id, g_vec in zip(movie_df['movie id'].values.tolist(), movie_df[genre_headers].values.tolist()):
            # check if movie_id was listed in ratings file and therefore in mapping dictionary
            if movie_id in v_dict.keys():
                v_features[v_dict[movie_id], :] = g_vec

        # user features

        sep = r'|'
        users_file = 'data/' + dataset + '/u.user'
        users_headers = ['user id', 'age', 'gender', 'occupation', 'zip code']
        users_df = pd.read_csv(users_file, sep=sep, header=None,
                               names=users_headers, engine='python')

        occupation = set(users_df['occupation'].values.tolist())

        age = users_df['age'].values
        age_max = age.max()

        gender_dict = {'M': 0., 'F': 1.}
        occupation_dict = {f: i for i, f in enumerate(occupation, start=2)}

        num_feats = 2 + len(occupation_dict)

        u_features = np.zeros((num_users, num_feats), dtype=np.float32)
        for _, row in users_df.iterrows():
            u_id = row['user id']
            if u_id in u_dict.keys():
                # age
                u_features[u_dict[u_id], 0] = row['age'] / np.float(age_max)
                # gender
                u_features[u_dict[u_id], 1] = gender_dict[row['gender']]
                # occupation
                u_features[u_dict[u_id], occupation_dict[row['occupation']]] = 1.

    

    u_features = sp.csr_matrix(u_features)
    v_features = sp.csr_matrix(v_features)

    print("User features shape: "+str(u_features.shape))
    print("Item features shape: "+str(v_features.shape))

    return u_features, v_features, rating_mx_train, train_labels, u_train_idx, v_train_idx, \
        val_labels, u_val_idx, v_val_idx, test_labels, u_test_idx, v_test_idx, class_values     

In [3]:
def map_data(data):
    """
    Map data to proper indices in case they are not in a continues [0, N) range
    Parameters
    ----------
    data : np.int32 arrays
    Returns
    -------
    mapped_data : np.int32 arrays
    n : length of mapped_data
    """
    uniq = list(set(data))

    id_dict = {old: new for new, old in enumerate(sorted(uniq))}
    data = np.array([id_dict[x] for x in data])
    n = len(uniq)

    return data, id_dict, n

In [4]:
def collate_movielens(data):
    g_list, label_list = map(list, zip(*data))
    g = dgl.batch(g_list)
    g_label = th.stack(label_list)
    return g, g_label

In [5]:
class MovieLens(object):
    """MovieLens dataset used by GCMC model
    """
    def __init__(self, data_name, testing=False, 
                 test_ratio=0.1, valid_ratio=0.2):
        (
                u_features, v_features, adj_train, train_labels, train_u_indices, train_v_indices,
                val_labels, val_u_indices, val_v_indices, test_labels, test_u_indices, 
                test_v_indices, class_values
            ) = load_official_trainvaltest_split(
                'ml-100k', testing, None, None, 1.0
            )
            
        self._num_user = u_features.shape[0]
        self._num_movie = v_features.shape[0]

        # reindex u and v, v nodes start after u
        train_v_indices += self.num_user
        val_v_indices += self.num_user
        test_v_indices += self.num_user

        self.train_rating_pairs = (th.LongTensor(train_u_indices), th.LongTensor(train_v_indices))
        self.valid_rating_pairs = (th.LongTensor(val_u_indices), th.LongTensor(val_v_indices))
        self.test_rating_pairs = (th.LongTensor(test_u_indices), th.LongTensor(test_v_indices))
        self.train_rating_values = th.FloatTensor(train_labels)
        self.valid_rating_values = th.FloatTensor(val_labels)
        self.test_rating_values = th.FloatTensor(test_labels)

        print("\tTrain rating pairs : {}".format(len(train_labels)))
        print("\tValid rating pairs : {}".format(len(val_labels)))
        print("\tTest rating pairs  : {}".format(len(test_labels)))

        # build dgl graph object, which is homogeneous and bidirectional and contains only training edges
        self.train_graph = dgl.graph((th.cat([self.train_rating_pairs[0], self.train_rating_pairs[1]]), 
                                      th.cat([self.train_rating_pairs[1], self.train_rating_pairs[0]])))
        self.train_graph.edata['etype'] = th.cat([self.train_rating_values, self.train_rating_values]).to(th.long)                    

    @property
    def num_rating(self):
        return self._rating.size

    @property
    def num_user(self):
        return self._num_user

    @property
    def num_movie(self):
        return self._num_movie


    def _generate_pair_value(self, rating_data):
        rating_pairs = (np.array([self._global_user_id_map[ele] for ele in rating_data["user_id"]],
                                 dtype=np.int32),
                        np.array([self._global_movie_id_map[ele] for ele in rating_data["movie_id"]],
                                 dtype=np.int32))
        # label ranges from 0. to 4.
        rating_values = rating_data["rating"].values.astype(np.float32) - 1.
        return rating_pairs[0], rating_pairs[1], rating_values

In [6]:
class MovieLensDataset(th.utils.data.Dataset):
    def __init__(self, links, g_labels, graph, 
                hop=1, sample_ratio=1.0, max_nodes_per_hop=200):
        self.links = links
        self.g_labels = g_labels
        self.graph = graph 

        self.hop = hop
        self.sample_ratio = sample_ratio
        self.max_nodes_per_hop = max_nodes_per_hop

    def __len__(self):
        return len(self.links[0])

    def __getitem__(self, idx):
        u, v = self.links[0][idx], self.links[1][idx]
        g_label = self.g_labels[idx]

        subgraph = subgraph_extraction_labeling(
            (u, v), self.graph, 
            hop=self.hop, sample_ratio=self.sample_ratio, max_nodes_per_hop=self.max_nodes_per_hop)

        return subgraph, g_label

In [7]:
movielens = MovieLens("ml-100k", testing=False)

train_dataset = MovieLensDataset(
        movielens.train_rating_pairs, movielens.train_rating_values, movielens.train_graph, 
        hop=1, sample_ratio=1.0, max_nodes_per_hop=200)
train_loader = th.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, 
                            num_workers=8, collate_fn=collate_movielens)

User features shape: (943, 23)
Item features shape: (1682, 18)
	Train rating pairs : 64000
	Valid rating pairs : 16000
	Test rating pairs  : 20000


In [8]:
def uniform(size, tensor):
    bound = 1.0 / math.sqrt(size)
    if tensor is not None:
        tensor.data.uniform_(-bound, bound)

In [9]:
def edge_drop(graph, edge_dropout=0.2, training=True):
    assert edge_dropout >= 0.0 and edge_dropout <= 1.0, 'Invalid dropout rate.'

    if not training:
        return graph

    # set edge mask to zero in directional mode
    src, _ = graph.edges()
    to_drop = src.new_full((graph.number_of_edges(), ), edge_dropout, dtype=th.float)
    to_drop = th.bernoulli(to_drop).to(th.bool)
    graph.edata['edge_mask'][to_drop] = 0

    return graph


In [10]:
class IGMC(nn.Module):
    # The GNN model of Inductive Graph-based Matrix Completion. 
    # Use RGCN convolution + center-nodes readout.
    
    def __init__(self, in_feats, gconv=RelGraphConv, latent_dim=[32, 32, 32, 32], 
                num_relations=5, num_bases=2, regression=False, edge_dropout=0.2, 
                force_undirected=False, side_features=False, n_side_features=0, 
                multiply_by=1):
        super(IGMC, self).__init__()

        self.regression = regression
        self.edge_dropout = edge_dropout
        self.force_undirected = force_undirected
        self.side_features = side_features
        self.multiply_by = multiply_by

        self.convs = th.nn.ModuleList()
        self.convs.append(gconv(in_feats, latent_dim[0], num_relations, num_bases=num_bases, self_loop=True, low_mem=True))
        for i in range(0, len(latent_dim)-1):
            self.convs.append(gconv(latent_dim[i], latent_dim[i+1], num_relations, num_bases=num_bases, self_loop=True, low_mem=True))
        
        self.lin1 = nn.Linear(2 * sum(latent_dim), 128)
        if side_features:
            self.lin1 = nn.Linear(2 * sum(latent_dim) + n_side_features, 128)
        if self.regression:
            self.lin2 = nn.Linear(128, 1)
        else:
            assert False
            # self.lin2 = nn.Linear(128, n_classes)
        self.reset_parameters()

    def reset_parameters(self):
        for conv in self.convs:
            size = conv.num_bases * conv.in_feat
            uniform(size, conv.weight)
            uniform(size, conv.w_comp)
            uniform(size, conv.loop_weight)
            uniform(size, conv.h_bias)
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    # @profile
    def forward(self, block):
        block = edge_drop(block, self.edge_dropout, self.training)

        concat_states = []
        x = block.ndata['x']
        for conv in self.convs:
            # edge mask zero denotes the edge dropped
            x = th.tanh(conv(block, x, block.edata['etype'], 
                             norm=block.edata['edge_mask'].unsqueeze(1)))
            concat_states.append(x)
        concat_states = th.cat(concat_states, 1)
        
        users = block.ndata['nlabel'][:, 0] == 1
        items = block.ndata['nlabel'][:, 1] == 1
        x = th.cat([concat_states[users], concat_states[items]], 1)
        # if self.side_features:
        #     x = th.cat([x, data.u_feature, data.v_feature], 1)

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        if self.regression:
            return x[:, 0] * self.multiply_by
        else:
            assert False
            # return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__

In [11]:
def evaluate(model, loader, device):
    # Evaluate RMSE
    model.eval()
    mse = 0.
    for batch in loader:
        with th.no_grad():
            preds = model(batch[0].to(device))
        labels = batch[1].to(device)
        mse += ((preds - labels) ** 2).sum().item()
    mse /= len(loader.dataset)
    return np.sqrt(mse)


In [12]:
def adj_rating_reg(model):
    arr_loss = 0
    for conv in model.convs:
        weight = conv.weight.view(conv.num_bases, conv.in_feat * conv.out_feat)
        weight = th.matmul(conv.w_comp, weight).view(conv.num_rels, conv.in_feat, conv.out_feat)
        arr_loss += th.sum((weight[1:, :, :] - weight[:-1, :, :])**2)
    return arr_loss

In [13]:
def train_epoch(model, loss_fn, optimizer, arr_lambda, loader, device, log_interval):
    model.train()

    epoch_loss = 0.
    iter_loss = 0.
    iter_mse = 0.
    iter_cnt = 0
    iter_dur = []

    for iter_idx, batch in enumerate(loader, start=1):
        t_start = time.time()

        inputs = batch[0].to(device)
        labels = batch[1].to(device)
        preds = model(inputs)
        loss = loss_fn(preds, labels).mean() + arr_lambda * adj_rating_reg(model)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * preds.shape[0]
        iter_loss += loss.item() * preds.shape[0]
        iter_mse += ((preds - labels) ** 2).sum().item()
        iter_cnt += preds.shape[0]
        iter_dur.append(time.time() - t_start)

        if iter_idx % log_interval == 0:
            print("Iter={}, loss={:.4f}, rmse={:.4f}, time={:.4f}".format(
                iter_idx, iter_loss/iter_cnt, iter_mse/iter_cnt, np.average(iter_dur)))
            iter_loss = 0.
            iter_mse = 0.
            iter_cnt = 0

    return epoch_loss / len(loader.dataset)

In [14]:
def train(args):
    ### prepare data and set model
    movielens = MovieLens(args.data_name, testing=args.testing,
                            test_ratio=args.data_test_ratio, valid_ratio=args.data_valid_ratio)
    if args.testing:
        test_dataset = MovieLensDataset(
            movielens.test_rating_pairs, movielens.test_rating_values, movielens.train_graph, 
            args.hop, args.sample_ratio, args.max_nodes_per_hop) 
    else:
        test_dataset = MovieLensDataset(
            movielens.valid_rating_pairs, movielens.valid_rating_values, movielens.train_graph, 
            args.hop, args.sample_ratio, args.max_nodes_per_hop)
    train_dataset = MovieLensDataset(
        movielens.train_rating_pairs, movielens.train_rating_values, movielens.train_graph, 
        args.hop, args.sample_ratio, args.max_nodes_per_hop)

    train_loader = th.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, 
                            num_workers=args.num_workers, collate_fn=collate_movielens)
    test_loader = th.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, 
                            num_workers=args.num_workers, collate_fn=collate_movielens)

    in_feats = (args.hop+1)*2 #+ movielens.train_graph.ndata['refex'].shape[1]
    model = IGMC(in_feats=in_feats, 
                 latent_dim=[32, 32, 32, 32],
                 num_relations=5, # movielens.num_rating, 
                 num_bases=4, 
                 regression=True, 
                 edge_dropout=args.edge_dropout,
                #  side_features=args.use_features,
                #  n_side_features=n_features,
                #  multiply_by=args.multiply_by
            ).to(args.device)
    loss_fn = nn.MSELoss().to(args.device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.train_lr, weight_decay=0)
    print("Loading network finished ...\n")

    ### prepare the logger
    logger = MetricLogger(args.save_dir, args.valid_log_interval)
    
    best_epoch = 0
    best_rmse = np.inf
    ### declare the loss information
    print("Start training ...")
    for epoch_idx in range(1, args.train_epochs+1):
        print ('Epoch', epoch_idx)
    
        train_loss = train_epoch(model, loss_fn, optimizer, args.arr_lambda, 
                                train_loader, args.device, args.train_log_interval)
        test_rmse = evaluate(model, test_loader, args.device)
        eval_info = {
            'epoch': epoch_idx,
            'train_loss': train_loss,
            'test_rmse': test_rmse,
        }
        print('=== Epoch {}, train loss {:.6f}, test rmse {:.6f} ==='.format(*eval_info.values()))

        if epoch_idx % args.train_lr_decay_step == 0:
            for param in optimizer.param_groups:
                param['lr'] = args.train_lr_decay_factor * param['lr']

        logger.log(eval_info, model, optimizer)
        if best_rmse > test_rmse:
            best_rmse = test_rmse
            best_epoch = epoch_idx
    eval_info = "Training ends. The best testing rmse is {:.6f} at epoch {}".format(best_rmse, best_epoch)
    print(eval_info)
    with open(os.path.join(args.save_dir, 'log.txt'), 'a') as f:
        f.write(eval_info)


In [15]:
def config():
    args = argparse.ArgumentParser(description='IGMC')
    # general settings
    args.testing = False
    args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    args.seed = 1234
    args.data_name = 'ml-100k'
    args.data_test_ratio = 0.1
    args.num_workers = 0
    args.data_valid_ratio = 0.2
    args.train_log_interval = 100
    args.valid_log_interval = 100
    args.save_appendix = 'debug'
    
    # subgraph extraction settings
    args.hop = 1
    args.sample_ratio = 1.0
    args.max_nodes_per_hop = 200
    
    # edge dropout settings
    args.edge_dropout = 0.2
    args.force_undirected = False
    
    # optimization settings
    args.train_lr = 1e-3
    args.train_min_lr = 1e-6
    args.train_lr_decay_factor = 0.1
    args.train_lr_decay_step = 50
    args.train_epochs = 80
    args.batch_size = 32
    args.arr_lambda = 0.001
    args.num_rgcn_bases = 4
   
    ## set save_dir according to localtime and test mode
    file_dir = os.path.dirname(os.path.realpath('__file__'))
    val_test_appendix = 'testmode' if args.testing else 'valmode'
    local_time = time.strftime('%y%m%d%H%M', time.localtime())
    args.save_dir = os.path.join(
        file_dir, 'log/{}_{}_{}_{}'.format(
            args.data_name, args.save_appendix, val_test_appendix, local_time
        )
    )
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir) 
    
    return args

In [16]:
args = config()

In [17]:
train(args)

User features shape: (943, 23)
Item features shape: (1682, 18)
	Train rating pairs : 64000
	Valid rating pairs : 16000
	Test rating pairs  : 20000
Loading network finished ...

Start training ...
Epoch 1
Iter=100, loss=1.4154, rmse=1.4127, time=0.0185
Iter=200, loss=1.1457, rmse=1.1430, time=0.0180
Iter=300, loss=1.0682, rmse=1.0656, time=0.0178
Iter=400, loss=1.0655, rmse=1.0630, time=0.0177
Iter=500, loss=1.0478, rmse=1.0454, time=0.0176
Iter=600, loss=1.0775, rmse=1.0753, time=0.0174
Iter=700, loss=1.0546, rmse=1.0524, time=0.0173
Iter=800, loss=1.0397, rmse=1.0376, time=0.0172
Iter=900, loss=1.0420, rmse=1.0400, time=0.0172
Iter=1000, loss=1.0683, rmse=1.0663, time=0.0171
Iter=1100, loss=1.0617, rmse=1.0598, time=0.0170
Iter=1200, loss=1.0590, rmse=1.0572, time=0.0170
Iter=1300, loss=1.0773, rmse=1.0756, time=0.0170
Iter=1400, loss=0.9967, rmse=0.9951, time=0.0169
Iter=1500, loss=1.0158, rmse=1.0142, time=0.0169
Iter=1600, loss=1.0127, rmse=1.0112, time=0.0169
Iter=1700, loss=1.021

Iter=1700, loss=0.8953, rmse=0.8933, time=0.0167
Iter=1800, loss=0.9154, rmse=0.9134, time=0.0167
Iter=1900, loss=0.9218, rmse=0.9198, time=0.0167
Iter=2000, loss=0.9359, rmse=0.9339, time=0.0167
=== Epoch 8, train loss 0.902128, test rmse 0.922923 ===
Epoch 9
Iter=100, loss=0.8758, rmse=0.8738, time=0.0166
Iter=200, loss=0.9145, rmse=0.9125, time=0.0167
Iter=300, loss=0.8669, rmse=0.8648, time=0.0167
Iter=400, loss=0.9255, rmse=0.9234, time=0.0167
Iter=500, loss=0.9174, rmse=0.9153, time=0.0167
Iter=600, loss=0.8560, rmse=0.8539, time=0.0167
Iter=700, loss=0.8963, rmse=0.8942, time=0.0167
Iter=800, loss=0.9220, rmse=0.9199, time=0.0167
Iter=900, loss=0.9091, rmse=0.9070, time=0.0167
Iter=1000, loss=0.9117, rmse=0.9095, time=0.0167
Iter=1100, loss=0.8949, rmse=0.8927, time=0.0167
Iter=1200, loss=0.8657, rmse=0.8633, time=0.0167
Iter=1300, loss=0.8933, rmse=0.8911, time=0.0167
Iter=1400, loss=0.8962, rmse=0.8938, time=0.0167
Iter=1500, loss=0.9146, rmse=0.9121, time=0.0167
Iter=1600, lo

Iter=1500, loss=0.8537, rmse=0.8509, time=0.0166
Iter=1600, loss=0.8745, rmse=0.8718, time=0.0166
Iter=1700, loss=0.8634, rmse=0.8607, time=0.0166
Iter=1800, loss=0.9121, rmse=0.9094, time=0.0166
Iter=1900, loss=0.8723, rmse=0.8696, time=0.0166
Iter=2000, loss=0.8635, rmse=0.8609, time=0.0166
=== Epoch 16, train loss 0.880841, test rmse 0.916412 ===
Epoch 17
Iter=100, loss=0.8399, rmse=0.8371, time=0.0167
Iter=200, loss=0.8658, rmse=0.8631, time=0.0168
Iter=300, loss=0.9193, rmse=0.9165, time=0.0167
Iter=400, loss=0.8625, rmse=0.8597, time=0.0167
Iter=500, loss=0.8747, rmse=0.8719, time=0.0167
Iter=600, loss=0.8958, rmse=0.8930, time=0.0167
Iter=700, loss=0.9304, rmse=0.9277, time=0.0167
Iter=800, loss=0.8792, rmse=0.8765, time=0.0167
Iter=900, loss=0.8521, rmse=0.8494, time=0.0167
Iter=1000, loss=0.8528, rmse=0.8501, time=0.0167
Iter=1100, loss=0.8706, rmse=0.8678, time=0.0167
Iter=1200, loss=0.8929, rmse=0.8901, time=0.0167
Iter=1300, loss=0.8988, rmse=0.8960, time=0.0167
Iter=1400, 

Iter=1300, loss=0.8967, rmse=0.8938, time=0.0166
Iter=1400, loss=0.8547, rmse=0.8518, time=0.0166
Iter=1500, loss=0.8814, rmse=0.8787, time=0.0166
Iter=1600, loss=0.8716, rmse=0.8689, time=0.0166
Iter=1700, loss=0.8948, rmse=0.8921, time=0.0166
Iter=1800, loss=0.8630, rmse=0.8602, time=0.0166
Iter=1900, loss=0.8318, rmse=0.8290, time=0.0166
Iter=2000, loss=0.9251, rmse=0.9224, time=0.0166
=== Epoch 24, train loss 0.880270, test rmse 0.918339 ===
Epoch 25
Iter=100, loss=0.8657, rmse=0.8630, time=0.0167
Iter=200, loss=0.8447, rmse=0.8418, time=0.0167
Iter=300, loss=0.8725, rmse=0.8698, time=0.0167
Iter=400, loss=0.8631, rmse=0.8602, time=0.0167
Iter=500, loss=0.8754, rmse=0.8726, time=0.0167
Iter=600, loss=0.8929, rmse=0.8900, time=0.0166
Iter=700, loss=0.8446, rmse=0.8417, time=0.0166
Iter=800, loss=0.8605, rmse=0.8576, time=0.0166
Iter=900, loss=0.8682, rmse=0.8653, time=0.0166
Iter=1000, loss=0.8836, rmse=0.8806, time=0.0167
Iter=1100, loss=0.8980, rmse=0.8950, time=0.0167
Iter=1200, 

Iter=1100, loss=0.8581, rmse=0.8551, time=0.0166
Iter=1200, loss=0.8956, rmse=0.8927, time=0.0166
Iter=1300, loss=0.8770, rmse=0.8740, time=0.0166
Iter=1400, loss=0.8505, rmse=0.8476, time=0.0166
Iter=1500, loss=0.8617, rmse=0.8590, time=0.0167
Iter=1600, loss=0.8996, rmse=0.8968, time=0.0167
Iter=1700, loss=0.8848, rmse=0.8819, time=0.0167
Iter=1800, loss=0.9025, rmse=0.8996, time=0.0167
Iter=1900, loss=0.8809, rmse=0.8780, time=0.0167
Iter=2000, loss=0.8836, rmse=0.8807, time=0.0167
=== Epoch 32, train loss 0.872481, test rmse 0.917430 ===
Epoch 33
Iter=100, loss=0.8657, rmse=0.8628, time=0.0169
Iter=200, loss=0.8548, rmse=0.8519, time=0.0168
Iter=300, loss=0.8686, rmse=0.8657, time=0.0168
Iter=400, loss=0.8676, rmse=0.8648, time=0.0168
Iter=500, loss=0.8608, rmse=0.8580, time=0.0167
Iter=600, loss=0.8745, rmse=0.8717, time=0.0167
Iter=700, loss=0.8600, rmse=0.8571, time=0.0167
Iter=800, loss=0.8771, rmse=0.8742, time=0.0167
Iter=900, loss=0.8832, rmse=0.8803, time=0.0167
Iter=1000, 

...

Iter=700, loss=0.8572, rmse=0.8541, time=0.0168
Iter=800, loss=0.8634, rmse=0.8600, time=0.0168
Iter=900, loss=0.8854, rmse=0.8821, time=0.0168
Iter=1000, loss=0.8756, rmse=0.8725, time=0.0168
Iter=1100, loss=0.8958, rmse=0.8928, time=0.0168
Iter=1200, loss=0.9132, rmse=0.9101, time=0.0168
Iter=1300, loss=0.8715, rmse=0.8684, time=0.0168
Iter=1400, loss=0.8836, rmse=0.8804, time=0.0168
Iter=1500, loss=0.8842, rmse=0.8811, time=0.0168
Iter=1600, loss=0.9026, rmse=0.8995, time=0.0168
Iter=1700, loss=0.8736, rmse=0.8706, time=0.0168
Iter=1800, loss=0.8974, rmse=0.8943, time=0.0168
Iter=1900, loss=0.8599, rmse=0.8568, time=0.0168
Iter=2000, loss=0.8974, rmse=0.8944, time=0.0168
=== Epoch 48, train loss 0.871480, test rmse 0.911461 ===
Epoch 49
Iter=100, loss=0.8532, rmse=0.8504, time=0.0166
Iter=200, loss=0.8823, rmse=0.8793, time=0.0165
Iter=300, loss=0.8556, rmse=0.8524, time=0.0165
Iter=400, loss=0.8713, rmse=0.8682, time=0.0165
Iter=500, loss=0.8826, rmse=0.8795, time=0.0165
Iter=600, 

Iter=500, loss=0.8241, rmse=0.8212, time=0.0166
Iter=600, loss=0.8443, rmse=0.8414, time=0.0165
Iter=700, loss=0.8589, rmse=0.8560, time=0.0166
Iter=800, loss=0.8653, rmse=0.8624, time=0.0166
Iter=900, loss=0.8456, rmse=0.8426, time=0.0166
Iter=1000, loss=0.8432, rmse=0.8403, time=0.0166
Iter=1100, loss=0.8327, rmse=0.8298, time=0.0166
Iter=1200, loss=0.8468, rmse=0.8439, time=0.0166
Iter=1300, loss=0.8489, rmse=0.8461, time=0.0166
Iter=1400, loss=0.8682, rmse=0.8653, time=0.0166
Iter=1500, loss=0.8529, rmse=0.8501, time=0.0166
Iter=1600, loss=0.8406, rmse=0.8378, time=0.0166
Iter=1700, loss=0.8593, rmse=0.8564, time=0.0166
Iter=1800, loss=0.8313, rmse=0.8285, time=0.0166
Iter=1900, loss=0.8660, rmse=0.8632, time=0.0166
Iter=2000, loss=0.8441, rmse=0.8412, time=0.0166
=== Epoch 56, train loss 0.845122, test rmse 0.906003 ===
Epoch 57
Iter=100, loss=0.8064, rmse=0.8035, time=0.0169
Iter=200, loss=0.8259, rmse=0.8230, time=0.0169
Iter=300, loss=0.8851, rmse=0.8822, time=0.0169
Iter=400, 

Iter=300, loss=0.8759, rmse=0.8730, time=0.0167
Iter=400, loss=0.8090, rmse=0.8062, time=0.0167
Iter=500, loss=0.8469, rmse=0.8440, time=0.0167
Iter=600, loss=0.8204, rmse=0.8176, time=0.0167
Iter=700, loss=0.8844, rmse=0.8815, time=0.0167
Iter=800, loss=0.8488, rmse=0.8459, time=0.0167
Iter=900, loss=0.8575, rmse=0.8546, time=0.0167
Iter=1000, loss=0.8355, rmse=0.8326, time=0.0167
Iter=1100, loss=0.8431, rmse=0.8403, time=0.0167
Iter=1200, loss=0.8461, rmse=0.8432, time=0.0167
Iter=1300, loss=0.8616, rmse=0.8587, time=0.0167
Iter=1400, loss=0.8503, rmse=0.8474, time=0.0167
Iter=1500, loss=0.8499, rmse=0.8471, time=0.0167
Iter=1600, loss=0.8196, rmse=0.8167, time=0.0167
Iter=1700, loss=0.8285, rmse=0.8256, time=0.0167
Iter=1800, loss=0.8348, rmse=0.8319, time=0.0167
Iter=1900, loss=0.8167, rmse=0.8139, time=0.0167
Iter=2000, loss=0.8301, rmse=0.8272, time=0.0167
=== Epoch 64, train loss 0.841301, test rmse 0.905209 ===
Epoch 65
Iter=100, loss=0.8554, rmse=0.8525, time=0.0169
Iter=200, 

Iter=100, loss=0.8369, rmse=0.8340, time=0.0167
Iter=200, loss=0.8181, rmse=0.8152, time=0.0167
Iter=300, loss=0.8301, rmse=0.8272, time=0.0167
Iter=400, loss=0.8522, rmse=0.8493, time=0.0167
Iter=500, loss=0.8444, rmse=0.8415, time=0.0167
Iter=600, loss=0.8322, rmse=0.8293, time=0.0167
Iter=700, loss=0.8569, rmse=0.8540, time=0.0167
Iter=800, loss=0.8505, rmse=0.8476, time=0.0167
Iter=900, loss=0.8508, rmse=0.8479, time=0.0167
Iter=1000, loss=0.8664, rmse=0.8635, time=0.0167
Iter=1100, loss=0.8209, rmse=0.8179, time=0.0167
Iter=1200, loss=0.8761, rmse=0.8732, time=0.0167
Iter=1300, loss=0.8401, rmse=0.8372, time=0.0167
Iter=1400, loss=0.8182, rmse=0.8153, time=0.0167
Iter=1500, loss=0.8164, rmse=0.8136, time=0.0167
Iter=1600, loss=0.8522, rmse=0.8493, time=0.0167
Iter=1700, loss=0.8199, rmse=0.8170, time=0.0167
Iter=1800, loss=0.8305, rmse=0.8276, time=0.0167
Iter=1900, loss=0.7980, rmse=0.7951, time=0.0167
Iter=2000, loss=0.8791, rmse=0.8762, time=0.0167
=== Epoch 72, train loss 0.83

=== Epoch 79, train loss 0.838899, test rmse 0.905581 ===
Epoch 80
Iter=100, loss=0.8237, rmse=0.8208, time=0.0167
Iter=200, loss=0.8262, rmse=0.8233, time=0.0166
Iter=300, loss=0.8391, rmse=0.8361, time=0.0166
Iter=400, loss=0.8328, rmse=0.8299, time=0.0166
Iter=500, loss=0.8435, rmse=0.8406, time=0.0166
Iter=600, loss=0.8528, rmse=0.8499, time=0.0166
Iter=700, loss=0.8110, rmse=0.8080, time=0.0166
Iter=800, loss=0.8316, rmse=0.8286, time=0.0166
Iter=900, loss=0.8291, rmse=0.8261, time=0.0165
Iter=1000, loss=0.8901, rmse=0.8872, time=0.0165
Iter=1100, loss=0.8686, rmse=0.8656, time=0.0165
Iter=1200, loss=0.8606, rmse=0.8576, time=0.0165
Iter=1300, loss=0.8354, rmse=0.8324, time=0.0165
Iter=1400, loss=0.8578, rmse=0.8548, time=0.0165
Iter=1500, loss=0.8073, rmse=0.8043, time=0.0165
Iter=1600, loss=0.8171, rmse=0.8142, time=0.0165
Iter=1700, loss=0.8366, rmse=0.8337, time=0.0165
Iter=1800, loss=0.8290, rmse=0.8261, time=0.0165
Iter=1900, loss=0.8293, rmse=0.8264, time=0.0165
Iter=2000, 